In [ ]:
!pip install geocoder

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 2.0 MB/s eta 0:00:00


In [ ]:
import requests
import pandas as pd
from geopy.geocoders import Nominatim
import geocoder

#got it using selenium
locations = ['Adambakkam', 'Adyar', 'Alandur', 'Alapakkam', 'Alwarpet', 'Alwarthirunagar', 'Ambattur', 'Aminjikarai', 'Anna Nagar', 'Annanur', 'Arumbakkam', 'Ashok Nagar', 'Avadi',
             'Ayanavaram', 'Beemannapettai', 'Besant Nagar', 'Basin Bridge', 'Chepauk', 'Chetput', 'Chintadripet', 'Chitlapakkam', 'Choolai', 'Choolaimedu', 'Chrompet', 'Egmore',
             'Ekkaduthangal', 'Eranavur', 'Ennore', 'Foreshore Estate', 'Fort St. George', 'George Town', 'Gopalapuram', 'Government Estate', 'Guindy', 'Guduvancheri', 'IIT Madras',
             'Injambakkam', 'ICF', 'Iyyapanthangal', 'Jafferkhanpet', 'Karapakkam', 'Kattivakkam', 'Kattupakkam', 'Kazhipattur', 'K.K. Nagar', 'Keelkattalai', 'Kattivakkam', 'Kilpauk',
             'Kodambakkam', 'Kodungaiyur', 'Kolathur', 'Korattur', 'Korukkupet', 'Kottivakkam', 'Kotturpuram', 'Kottur', 'Kovilambakkam', 'Koyambedu', 'Kundrathur', 'Madhavaram',
             'Madhavaram Milk Colony', 'Madipakkam', 'Madambakkam', 'Maduravoyal', 'Manali', 'Manali New Town', 'Manapakkam', 'Mandaveli', 'Mangadu', 'Mannady', 'Mathur', 'Medavakkam',
             'Meenambakkam', 'MGR Nagar', 'Minjur', 'Mogappair', 'MKB Nagar', 'Mount Road', 'Moolakadai', 'Moulivakkam', 'Mugalivakkam', 'Mudichur', 'Mylapore', 'Nandanam', 'Nanganallur',
             'Nanmangalam', 'Neelankarai', 'Nemilichery', 'Nesapakkam', 'Nolambur', 'Noombal', 'Nungambakkam', 'Otteri', 'Padi', 'Pakkam', 'Palavakkam', 'Pallavaram', 'Pallikaranai', 'Pammal',
             'Park Town', "Parry's Corner", 'Pattabiram', 'Pattaravakkam', 'Pazhavanthangal', 'Peerkankaranai', 'Perambur', 'Peravallur', 'Perumbakkam', 'Perungalathur', 'Perungudi', 'Pozhichalur',
             'Poonamallee', 'Porur', 'Pudupet', 'Pulianthope', 'Purasaiwalkam', 'Puthagaram', 'Puzhal', 'Puzhuthivakkam/ Ullagaram', 'Raj Bhavan', 'Ramavaram', 'Red Hills', 'Royapettah', 'Royapuram',
             'Saidapet', 'Saligramam', 'Santhome', 'Sembakkam', 'Selaiyur', 'Shenoy Nagar', 'Sholavaram', 'Sholinganallur', 'Sithalapakkam', 'Sowcarpet', 'St.Thomas Mount', 'Surapet', 'Tambaram',
             'Teynampet', 'Tharamani', 'T. Nagar', 'Thirumangalam', 'Thirumullaivoyal', 'Thiruneermalai', 'Thiruninravur', 'Thiruvanmiyur', 'Tiruverkadu', 'Thiruvotriyur', 'Thuraipakkam', 'Tirusulam',
             'Tiruvallikeni', 'Tondiarpet', 'United India Colony', 'Vandalur', 'Vadapalani', 'Valasaravakkam', 'Vallalar Nagar', 'Vanagaram', 'Velachery', 'Velappanchavadi', 'Villivakkam', 'Virugambakkam',
             'Vyasarpadi', 'Washermanpet', 'West Mambalam']
print(len(locations))
kl_df = pd.DataFrame({"Neighborhood": locations})

address = 'Chennai, India'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chennai, India {}, {}.'.format(latitude, longitude))

def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Chennai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords
# Call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(location) for location in kl_df["Neighborhood"].tolist()]
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
# Merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']
print(kl_df.shape)
kl_df.head()

164
The geograpical coordinate of Chennai, India 13.0836939, 80.270186.
(164, 3)


,Neighborhood,Latitude,Longitude
0,Adambakkam,12.99192,80.20603
1,Adyar,13.00303,80.25186
2,Alandur,13.00013,80.20051
3,Alapakkam,13.04611,80.16499
4,Alwarpet,13.03473,80.25415


import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_0, *['index'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_1, *['Latitude'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_2, *['Longitude'], **{})
chart

...

import numpy as np
from google.colab import autoviz

def violin_plot(df, value_colname, facet_colname, figscale=1, mpl_palette_name='Dark2', **kwargs):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (12 * figscale, 1.2 * figscale * len(df[facet_colname].unique()))
  plt.figure(figsize=figsize)
  sns.violinplot(df, x=value_colname, y=facet_colname, palette=mpl_palette_name, **kwargs)
  sns.despine(top=True, right=True, bottom=True, left=True)
  return autoviz.MplChart.from_current_mpl_state()

chart = violin_plot(_df_23, *['Latitude', 'Neighborhood'], **{'inner': 'stick'})
chart

import numpy as np
from google.colab import autoviz

def violin_plot(df, value_colname, facet_colname, figscale=1, mpl_palette_name='Dark2', **kwargs):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (12 * figscale, 1.2 * figscale * len(df[facet_colname].unique()))
  plt.figure(figsize=figsize)
  sns.violinplot(df, x=value_colname, y=facet_colname, palette=mpl_palette_name, **kwargs)
  sns.despine(top=True, right=True, bottom=True, left=True)
  return autoviz.MplChart.from_current_mpl_state()

chart = violin_plot(_df_24, *['Longitude', 'Neighborhood'], **{'inner': 'stick'})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_25, *['index', 'Latitude', 'Neighborhood'], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_26, *['index', 'Longitude', 'Neighborhood'], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_27, *['index', 'count()', 'Neighborhood'], **{})
chart

In [ ]:
import folium
#map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)
loc=str(input("Enter location: "))
for i in range(len(locations)):
  if(loc==locations[i]):
    coords=get_latlng(locations[i])
    label='{}'.format(locations[i])
    lat=coords[0]
    lng=coords[1]
map_kl = folium.Map(location=[lat, lng], zoom_start=11)
label = folium.Popup(label, parse_html=True)
folium.CircleMarker([lat, lng],radius=7,popup=label,color='red',fill=True,fill_color='#3186cc',fill_opacity=1).add_to(map_kl)
map_kl

Enter location: Tirusulam


In [ ]:
import pandas as pd # for data processing
import folium   #for creating maps
import requests  #for retreiving Information from URL
from geopy.geocoders import Nominatim  #converting address to cordinates
from pandas.io.json import json_normalize
import json

def list2string(s):
  str1=""
  for e in s:
    str1+=e
  return str1


radius = 2000
LIMIT = 50

URL= "https://api.foursquare.com/v3/places/search?ll={},{}&radius={}&limit={}"
headers = {
      "Accept": "application/json",
      "Authorization": "xxxx" #key is kept a secret
    }

df_list=[]

for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
      url = URL.format(lat, lng, radius, LIMIT)
      results = requests.request("GET", url, headers=headers).json()
      #print(results)

      for each_result in results['results']: # filter the result based on JSON identification
            result={}
            result['Neighborhood']=neighborhood
            result['Neighborhood Latitude']=lat
            result['Neighborhood Longitude']=lng
            result['Name']=each_result['name']
            result['Venue Latitude']=each_result['geocodes']['main']['latitude']
            result['Venue Longitude']=each_result['geocodes']['main']['longitude']
            result['Category_Names']=list2string([each_name['name'] for each_name in each_result['categories']])
            df_list.append(result.copy())

venues_df=pd.DataFrame(df_list) # return dataframe
venues_df.columns=['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude','VenueCategory']
print(venues_df.shape)
venues_df.head()

(4709, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Adambakkam,12.99192,80.20603,Pizza Republic,12.991098,80.198625,Pizzeria
1,Adambakkam,12.99192,80.20603,Imax,12.992246,80.217300,Movie Theater
2,Adambakkam,12.99192,80.20603,Luxe Cinemas,12.991168,80.216830,Movie Theater
3,Adambakkam,12.99192,80.20603,Sukkkubai Beef Biryani Shop,12.998756,80.201522,Indian Restaurant
4,Adambakkam,12.99192,80.20603,Hotel Mass,12.998963,80.207650,Restaurant


In [ ]:
venues_df.groupby(["Neighborhood"]).count()
venues_df['VenueCategory'].unique()


array(['Pizzeria', 'Movie Theater', 'Indian Restaurant', 'Restaurant',
       'Business and Professional Services', 'Chinese Restaurant',
       'Church', 'Retail', 'DeliFast Food Restaurant', 'Bakery', 'Temple',
       'Spiritual Center', 'Ice Cream Parlor', 'Housing Development',
       'Vegan and Vegetarian Restaurant', 'Coffee Shop',
       'Fast Food Restaurant',
       'Indian RestaurantVegan and Vegetarian Restaurant', 'Snack Place',
       'ChurchTravel Agency', 'Bicycle Store', 'Swimming Pool',
       'BBQ Joint', 'Italian Restaurant', 'Playground', 'Sandwich Spot',
       'Badminton Court', 'Seafood Restaurant', 'Education',
       'Miscellaneous Store', "Women's Store",
       'PizzeriaFast Food Restaurant', 'Burger JointFast Food Restaurant',
       'Dessert ShopFast Food Restaurant', 'North Indian Restaurant',
       'Asian Restaurant', "Clothing StoreMen's Store",
       'Ice Cream ParlorJuice BarFruit and Vegetable Store', 'BistroCafé',
       'Dining and Drinking', 'Mov

In [ ]:
map_kl = folium.Map(location=[lat, lng], zoom_start=11)
# Adding markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'],  kl_df['Longitude'], kl_df['Neighborhood']):
 label = '{}'.format(neighborhood)
 label = folium.Popup(label, parse_html=True)
 folium.CircleMarker([lat, lng],radius=7,popup=label,color='blue',fill=True,fill_color='#3186cc',fill_opacity=0.7).add_to(map_kl)
map_kl

In [ ]:
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")
kl_onehot['Neighborhoods'] = venues_df['Neighborhood']
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]
kl_grouped=kl_onehot.groupby(["Neighborhoods"]).sum().reset_index()
print(kl_grouped.shape)
kl_grouped

(159, 282)


,Neighborhoods,,Afghan Restaurant,African Restaurant,American Restaurant,Amusement Park,Antique Store,Aquarium,Arcade,Arts and Entertainment,...,Theater,Tobacco Store,Trade School,Transport Hub,Udupi Restaurant,Vegan and Vegetarian Restaurant,Vegan and Vegetarian RestaurantLodging,Video Store,Women's Store,Zoo
0,Adambakkam,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,Adyar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
2,Alandur,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,Alapakkam,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,Alwarpet,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,Villivakkam,1,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1
155,Virugambakkam,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
156,Vyasarpadi,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
157,Washermanpet,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


**K-Means Clustering**

*   Uses classes from 0,1,2,3
*   Highest number of restaurants denoted in cluster 3



In [ ]:
from sklearn.cluster import KMeans
import numpy as np
from matplotlib import cm,colors
from time import time
#getting the time
st= time()
len((kl_grouped[kl_grouped["Vegan and Vegetarian Restaurant"] > 0]))
kl_rest = kl_grouped[["Neighborhoods","Vegan and Vegetarian Restaurant"]]
kclusters=4
kl_clustering = kl_rest.drop(["Neighborhoods"], 1)
kmeans = KMeans(n_clusters=kclusters,random_state=0).fit(kl_clustering)
kl_merged = kl_rest.copy()
kl_merged["Cluster Labels"] = kmeans.labels_
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head(10)
kl_merged['Latitude'] = kl_df['Latitude']
kl_merged['Longitude'] = kl_df['Longitude']
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged.head()
map_clusters = folium.Map(location=[13.0836939, 80.270186], zoom_start=11)
# Setting color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
  label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
  folium.CircleMarker([lat,lon],radius=5,popup=label,color=rainbow[cluster],fill=True,fill_color=rainbow[cluster-1],fill_opacity=0.7).add_to(map_clusters)
et=time()
el_t=et-st
print("Elapsed time: ", el_t)
map_clusters

<ipython-input-62-887294d72675>:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  kl_clustering = kl_rest.drop(["Neighborhoods"], 1)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Elapsed time:  0.044599056243896484


In [ ]:
print(rainbow)

['#8000ff', '#2adddd', '#d4dd80', '#ff0000']


In [ ]:
kl_merged["Cluster Labels"]

0      0
47     0
50     0
51     0
53     0
      ..
55     2
11     2
43     2
158    2
135    3
Name: Cluster Labels, Length: 159, dtype: int32

In [ ]:
from sklearn.cluster import KMeans
import numpy as np
from matplotlib import cm,colors
len((kl_grouped[kl_grouped["Vegan and Vegetarian Restaurant"] > 0]))
kl_rest = kl_grouped[["Neighborhoods","Vegan and Vegetarian Restaurant"]]
tot_rest=sum(kl_grouped["Vegan and Vegetarian Restaurant"])
print(tot_rest)

71


In [ ]:
with pd.option_context('display.max_rows', None,):
   print(kl_rest)

                 Neighborhoods  Vegan and Vegetarian Restaurant
0                   Adambakkam                                1
1                        Adyar                                1
2                      Alandur                                1
3                    Alapakkam                                0
4                     Alwarpet                                1
5              Alwarthirunagar                                0
6                     Ambattur                                1
7                  Aminjikarai                                0
8                   Anna Nagar                                2
9                      Annanur                                0
10                  Arumbakkam                                2
11                 Ashok Nagar                                2
12                       Avadi                                0
13                  Ayanavaram                                0
14                Basin Bridge          

In [ ]:
kl_rest["Contribution (in %)"]=(kl_rest["Vegan and Vegetarian Restaurant"]/kl_rest["Vegan and Vegetarian Restaurant"].sum())*100
pd.option_context('display.max_rows', None,)
kl_rest.to_csv('/content/drive/MyDrive/Colab Notebooks/table.csv', index=False)

<ipython-input-50-04bffdd7e458>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kl_rest["Contribution (in %)"]=(kl_rest["Vegan and Vegetarian Restaurant"]/kl_rest["Vegan and Vegetarian Restaurant"].sum())*100


In [ ]:
kclusters=4
kl_clustering = kl_rest.drop(["Neighborhoods"], 1)
kmeans = KMeans(n_clusters=kclusters,random_state=0).fit(kl_clustering)
kl_merged = kl_rest.copy()
kl_merged["Cluster Labels"] = kmeans.labels_
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head(10)

<ipython-input-24-394103b8b426>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  kl_clustering = kl_rest.drop(["Neighborhoods"], 1)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,Neighborhood,Vegan and Vegetarian Restaurant,Cluster Labels
0,Adambakkam,1,0
1,Adyar,1,0
2,Alandur,1,0
3,Alapakkam,0,1
4,Alwarpet,1,0
5,Alwarthirunagar,0,1
6,Ambattur,1,0
7,Aminjikarai,0,1
8,Anna Nagar,2,2
9,Annanur,0,1


**WITH DBSCAN**

*   Returns classes from -1,0,1,2
*   Using 5 cluster types, (-1 has the most)



In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np
from matplotlib import cm,colors
from time import time
st=time()
len((kl_grouped[kl_grouped["Vegan and Vegetarian Restaurant"] > 0]))
kl_rest = kl_grouped[["Neighborhoods","Vegan and Vegetarian Restaurant"]]
kclusters=5
kl_clustering = kl_rest.drop(["Neighborhoods"], 1)
dbscan = DBSCAN(eps=0.1, min_samples=4)
kl_merged = kl_rest.copy()
kl_merged["Cluster Labels"] = dbscan.fit_predict(kl_clustering)
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged['Latitude'] = kl_df['Latitude']
kl_merged['Longitude'] = kl_df['Longitude']
kl_merged.sort_values(["Cluster Labels"], inplace=True)
map_clusters = folium.Map(location=[13.0836939, 80.270186], zoom_start=11)
# Setting color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
  label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
  folium.CircleMarker([lat,lon],radius=5,popup=label,color=rainbow[cluster-1],fill=True,fill_color=rainbow[cluster-1],fill_opacity=0.7).add_to(map_clusters)
et=time()
el_t=et-st
print("Elapsed time: ", el_t)
map_clusters

<ipython-input-17-c9f48b0365f0>:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  kl_clustering = kl_rest.drop(["Neighborhoods"], 1)


Elapsed time:  0.03510308265686035


In [ ]:
print(kl_merged["Cluster Labels"])

25    -1
0      0
87     0
86     0
84     0
      ..
152    3
95     3
125    3
130    3
81     3
Name: Cluster Labels, Length: 163, dtype: int64


In [ ]:
#Accuracy metrics
from sklearn.metrics import silhouette_score
silhouette_score(kl_clustering)

-0.22562949737098956

In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np
from matplotlib import cm,colors
len((kl_grouped[kl_grouped["Vegan and Vegetarian Restaurant"] > 0]))
kl_rest = kl_grouped[["Neighborhoods","Vegan and Vegetarian Restaurant"]]
kl_rest

,Neighborhoods,Vegan and Vegetarian Restaurant
0,Adambakkam,0
1,Adyar,0
2,Alandur,0
3,Alapakkam,1
4,Alwarpet,0
...,...,...
158,Villivakkam,0
159,Virugambakkam,1
160,Vyasarpadi,0
161,Washermanpet,2


In [ ]:
kl_clustering = kl_rest.drop(["Neighborhoods"], 1)
kl_clustering

<ipython-input-27-98de2071ef18>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  kl_clustering = kl_rest.drop(["Neighborhoods"], 1)


,Vegan and Vegetarian Restaurant
0,0
1,0
2,0
3,1
4,0
...,...
158,0
159,1
160,0
161,2


In [ ]:
dbscan = DBSCAN(eps=0.1, min_samples=4)
kl_merged = kl_rest.copy()
kl_merged["Cluster Labels"] = dbscan.fit_predict(kl_clustering)
kl_merged
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head(10)

,Neighborhood,Vegan and Vegetarian Restaurant,Cluster Labels
0,Adambakkam,0,0
1,Adyar,0,0
2,Alandur,0,0
3,Alapakkam,1,1
4,Alwarpet,0,0
5,Alwarthirunagar,1,1
6,Ambattur,0,0
7,Aminjikarai,0,0
8,Anna Nagar,0,0
9,Annanur,0,0


In [ ]:
kl_merged['Latitude'] = kl_df['Latitude']
kl_merged['Longitude'] = kl_df['Longitude']
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged.head()

,Neighborhood,Vegan and Vegetarian Restaurant,Cluster Labels,Latitude,Longitude
39,Jafferkhanpet,3,-1,13.02954,80.20528
130,Sholinganallur,3,-1,13.24119,80.16036
11,Ashok Nagar,3,-1,13.03550,80.21218
160,Vyasarpadi,0,0,13.05591,80.19346
4,Alwarpet,0,0,13.03473,80.25415


In [ ]:
map_clusters = folium.Map(location=[13.0836939, 80.270186], zoom_start=11)
# Setting color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
  label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
  folium.CircleMarker([lat,lon],radius=5,popup=label,color=rainbow[cluster-1],fill=True,fill_color=rainbow[cluster-1],fill_opacity=0.7).add_to(map_clusters)
map_clusters

In [ ]:
print(kl_rest['Vegan and Vegetarian Restaurant'])

0      0
1      0
2      0
3      1
4      0
      ..
158    0
159    0
160    0
161    1
162    1
Name: Vegan and Vegetarian Restaurant, Length: 163, dtype: uint8


**WITH GAUSSIAN MIXTURE MODEL**

*   Same as K-Means, uses classes from 0-3
*   3 is the highest number of hotels



In [ ]:
from sklearn.mixture import GaussianMixture
import numpy as np
from matplotlib import cm,colors
from time import time
st=time()
len((kl_grouped[kl_grouped["Vegan and Vegetarian Restaurant"] > 0]))
kl_rest = kl_grouped[["Neighborhoods","Vegan and Vegetarian Restaurant"]]
kclusters=4
kl_clustering = kl_rest.drop(["Neighborhoods"], 1)
gmm=GaussianMixture(n_components=4, random_state=0)
kl_merged = kl_rest.copy()
kl_merged["Cluster Labels"] = gmm.fit_predict(kl_clustering)
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged['Latitude'] = kl_df['Latitude']
kl_merged['Longitude'] = kl_df['Longitude']
kl_merged.sort_values(["Cluster Labels"], inplace=True)
map_clusters = folium.Map(location=[13.0836939, 80.270186], zoom_start=11)
# Setting color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
  label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
  folium.CircleMarker([lat,lon],radius=5,popup=label,color=rainbow[cluster-1],fill=True,fill_color=rainbow[cluster-1],fill_opacity=0.7).add_to(map_clusters)
et=time()
el_t=et-st
print("Elapsed time: ", el_t)
map_clusters

<ipython-input-23-ec7038f1d67b>:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  kl_clustering = kl_rest.drop(["Neighborhoods"], 1)


Elapsed time:  0.19018006324768066


In [ ]:
print(kl_merged["Cluster Labels"])

162    0
103    0
72     0
104    0
70     0
      ..
102    2
81     2
11     3
130    3
39     3
Name: Cluster Labels, Length: 163, dtype: int64


With this, we can conclude that other than the cluster markings changing for DBSCAN, everything else returns similar results.